In [8]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from matplotlib import pyplot
import numpy as np
import os
os.environ['NUMBA_DISABLE_JIT'] = '1'

In [10]:
from pystrict import strict
import numpy as np
from PySDM.physics.constants import si
from PySDM.initialisation.spectra import Gamma
from PySDM.dynamics.collisions.collision_kernels import Golovin
from PySDM.dynamics.collisions.coalescence_efficiencies import ConstEc
from PySDM.dynamics.collisions.breakup_efficiencies import ConstEb
from PySDM.dynamics.collisions.breakup_fragmentations import Feingold1988Frag, ExponFrag, AlwaysN
from PySDM.formulae import Formulae
from PySDM.physics.constants_defaults import rho_w


In [11]:
@strict
class Settings:

    def __init__(self):
        self.formulae = Formulae()
        self.n_sd = 2**4
        self.n_part = 1e4 / si.cm**3
        self.theta = 0.33e-9 * si.g / rho_w
        self.k = 1
        self.dv = 1 * si.m**3
        self.norm_factor = self.n_part * self.dv
        self.dt = 1 * si.seconds
        self.adaptive = False
        self.seed = 44
        self._steps = [i for i in range(60)] #[0, 60]
        #self.kernel = Golovin(b=2000 * si.cm**3 / si.g / si.s * rho_w)
        self.kernel = Golovin(b=20000 * si.cm**3 / si.g / si.s * rho_w)
        self.coal_effs = [ConstEc(Ec=0.8), ConstEc(Ec=0.9),ConstEc(Ec=1.0)]
        self.rho = rho_w
        self.vmin = 0.1*si.um**3
        self.fragtol = 1e-3
        #self.fragmentation = Feingold1988Frag(scale=self.k * self.theta / 2, fragtol=self.fragtol, vmin=self.vmin)
        self.fragmentation = ExponFrag(scale=self.k*self.theta/2)
        #self.fragmentation = AlwaysN(n=2)
        self.break_eff = ConstEb(1.0) # no "bouncing"
        self.spectrum = Gamma(norm_factor=self.norm_factor, k=self.k, theta=self.theta)
        # self.radius_bins_edges = np.logspace(
        #     np.log10(1 * si.um),
        #     np.log10(1000 * si.um),
        #     num=128, endpoint=True
        # )
        # self.radius_range = [0 * si.um, 1e6 * si.um]


    @property
    def output_steps(self):
        return [int(step/self.dt) for step in self._steps]

In [12]:
from PySDM.backends import CPU
from PySDM.builder import Builder
from PySDM.environments import Box
from PySDM.dynamics import Collision
from PySDM.initialisation.sampling.spectral_sampling import ConstantMultiplicity
import PySDM.products.size_spectral.arbitrary_moment as am

In [13]:
settings = Settings()
coal_eff = ConstEc(Ec=0.0)
backend = CPU
builder = Builder(n_sd=settings.n_sd, backend=backend(settings.formulae))
env = Box(dv=settings.dv, dt=settings.dt)
builder.set_environment(env)
env['rhod'] = 1.0
attributes = {}
attributes['volume'], attributes['n'] = \
    ConstantMultiplicity(settings.spectrum).sample(settings.n_sd)
collision = Collision(
    settings.kernel,
    coal_eff,
    settings.break_eff,
    settings.fragmentation,
    adaptive=settings.adaptive
)
builder.add_dynamic(collision)
M0 = am.make_arbitrary_moment_product(rank=0, attr='volume', attr_unit='m^3')
M1 = am.make_arbitrary_moment_product(rank=1, attr='volume', attr_unit='m^3')
M2 = am.make_arbitrary_moment_product(rank=2, attr='volume', attr_unit='m^3')
products = (
    M0(name='M0'),
    M1(name='M1'),
    M2(name='M2')
)
particulator = builder.build(attributes, products)

In [14]:
t = settings.output_steps
moments = np.zeros((3, len(t)))
j = 0
for step in settings.output_steps:
    particulator.run(step - particulator.n_steps)
    moments[:,j] = [particulator.products['M0'].get()[0], particulator.products['M1'].get()[0], 
        particulator.products['M2'].get()[0]]
    j += 1
    print(particulator.attributes['n'].to_ndarray())
fig, ax = pyplot.subplots(1, 3)
ax[0].plot(t,moments[0,:])
ax[1].plot(t,moments[1,:]*moments[0,:])
ax[2].plot(t,moments[2,:]*moments[0,:])

[624987500 624987500 624987500 624987500 624987500 624987500 624987500
 624987500 624987500 624987500 624987500 624987500 624987500 624987500
 624987500 624987500]
[ 1.          3.52393926 17.33280638 11.73905134  1.         27.33046357
  4.96888267  2.95533753]
[ 624987500  624987500  624987500  624987500  624987500  624987500
  624987500  624987500  624987500  624987500 8540599049 8540599049
  624987500  624987500  923524507  923524507]
[20.74351813  2.2214341  18.75817935  1.         21.04688068  1.
  6.36160791  3.8131608 ]
[ 624987500  624987500 7915611549 1388368546  624987500  624987500
 7915611549  624987500  624987500  624987500  624987500  624987500
  624987500  624987500  923524507  923524507]
[  5.36526178   8.25461198 119.43241915   1.71759097   1.95071678
   1.           1.           1.4400916 ]
[ 624987500  624987500  923524507  624987500  624987500  624987500
 7915611549  624987500  923524507  624987500 7290624049  624987500
  624987500  624987500 1388368546  624987500]

Exception: Breakup step failed due to multiplicity exceeding maximum allowed